<a href="https://colab.research.google.com/github/AyozeGS/Evolutionary-algorithm---Traveling-Salesman-Problem.ipynb/blob/main/Evolutionary_algorithm_Traveling_Salesman_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VIAJANTE DE COMERCIO CON ALGORITMO EVOLUTIVO

Algoritmo que resuelve el problema de encontrar un camino lo más óptimo posible usando el algoritmo genético cuyas funciones de mutación y de recombinación vienen descritas en https://www.youtube.com/watch?v=3Kzj2FNaua8

La tabla no usa distnacias

In [ ]:
import pandas as pd
import numpy as np
import random
from tabulate import tabulate

# Carga y limpieza de los datos del fichero csv.
df = pd.read_csv("data_municipios2.csv", index_col=0)
df[df.isna()] = 0
display(df)

,Agaete,Agüimes,La Aldea de San Nicolás,Artenara,Arucas,Firgas,Gáldar,Ingenio,Mogán,Moya,...,San Bartolomé de Tirajana,Santa Brígida,Santa Lucía de Tirajana,Guía,Tejeda,Telde,Teror,Valleseco,Valsequillo,Vega de San Mateo
Distancia(km),,,,,,,,,,,,,,,,,,,,,
Agaete,0.0,64.0,30.0,34.0,28.0,31.0,9.0,62.0,53.0,25.0,...,61.0,43.0,85.0,11.0,42.0,50.0,37.0,30.0,60.0,51.0
Agüimes,65.0,0.0,62.0,47.0,40.0,46.0,57.0,3.0,57.0,57.0,...,28.0,34.0,23.0,56.0,37.0,21.0,46.0,37.0,34.0,42.0
La Aldea de San Nicolás,30.0,62.0,0.0,29.0,57.0,61.0,38.0,60.0,25.0,54.0,...,39.0,72.0,45.0,40.0,35.0,79.0,56.0,60.0,89.0,47.0
Artenara,34.0,47.0,29.0,0.0,33.0,23.0,33.0,45.0,54.0,27.0,...,40.0,30.0,46.0,30.0,8.0,49.0,27.0,37.0,40.0,28.0
Arucas,27.0,40.0,56.0,32.0,0.0,7.0,19.0,38.0,89.0,18.0,...,67.0,19.0,61.0,17.0,30.0,26.0,11.0,6.0,36.0,27.0
Firgas,32.0,46.0,61.0,23.0,8.0,0.0,23.0,44.0,95.0,23.0,...,45.0,25.0,67.0,22.0,27.0,32.0,12.0,12.0,42.0,32.0
Gáldar,9.0,57.0,38.0,33.0,20.0,24.0,0.0,54.0,105.0,17.0,...,58.0,35.0,78.0,3.0,39.0,42.0,29.0,23.0,52.0,43.0
Ingenio,63.0,3.0,60.0,45.0,37.0,44.0,54.0,0.0,60.0,54.0,...,31.0,32.0,25.0,53.0,35.0,18.0,44.0,34.0,31.0,39.0
Mogán,54.0,57.0,25.0,54.0,89.0,95.0,106.0,61.0,0.0,105.0,...,53.0,83.0,66.0,104.0,39.0,69.0,95.0,85.0,82.0,90.0


# Funciones

In [ ]:
# Fitness: function to evaluate solution. It uses sum of distances
def fitness(member, data_matrix):

  sum = 0
  for i in range(len(member)-1):
    idx_row = member[i]
    idx_col  = member[i+1]
    sum += data_matrix.iloc[idx_row][idx_col]
  return sum

In [ ]:
# Mutation: function to apply a little change in a member in order to create other
def mutate(member):

  # Get 2 randoms index for exchange their values
  mutated_member = np.copy(member)
  mutated_index_1 = random.randint(0, len(member)-1)
  mutated_index_2 = random.randint(0, len(member)-1)
  while mutated_index_2 == mutated_index_1:
    mutated_index_2 = random.randint(0, len(member)-1)

  # Exchange values
  mutated_member[[mutated_index_1, mutated_index_2]] = mutated_member[[mutated_index_2, mutated_index_1]]
  return mutated_member


# Function to generate multiple mutated members
def mutate_generation(member, n):

  #If n = 0 return empty array
  if not n:
    return np.array([], dtype=np.int64).reshape(0,len(member))

  mutated_members = []
  mutated_members.append(mutate(member))

  while len(mutated_members) < n:
    mutated_member = mutate(member)
    #Check if member already in population
    if not (mutated_member == mutated_members).all(axis=1).any():
      mutated_members.append(mutated_member)
  return np.array(mutated_members)

In [ ]:
# Recombination : function to combine two members in order to create a new one
def recombinate(member1, member2):

  recombinated_member = np.copy(member1)
  # Reorder one member values using the other one as indexes.
  for idx, v in enumerate(recombinated_member):
    recombinated_member[idx] = member2[member1[idx]]

  return recombinated_member


# Function to generate multiple recombinated members
def recombinate_members(members, n):
  recombinated_members = []

  #If n = 0 return empty array
  if not n:
    return np.array([], dtype=np.int64).reshape(0,len(members[0]))

  for member1 in members:
    for member2 in members:
      # Self-recombinateion is ignored
      if not (member1 == member2).all():
         recombinated_members.append(recombinate(member1, member2))
  # Return n first recombinated members.
  return np.array(recombinated_members[:n])

In [ ]:
# Selection: Choice best members to use in next generation
def selection(population, n, dataframe):

  #If n = 0 return empty array
  if not n:
    return np.array([], dtype=np.int64).reshape(0,len(population[0]))

  population = np.array(sorted(population, key= lambda member: fitness(member, dataframe)))
  return population[:n]

In [ ]:
# Generate n random members
def generate_random_members(n, size):

  #If n = 0 return empty array
  if not n:
    return np.array([], dtype=np.int64).reshape(0,size)

  indexes = range(0,size)
  # reorder indexes Ex: [0 1 2 3 4 5 6] -> [4 0 5 2 6 1 3]
  population = np.array([np.random.permutation(indexes)])
  while len(population) < n:
    member = np.random.permutation(indexes)
    # Check repeated members
    if not (member == population).all(axis=1).any():
      population = np.append(population, [member], axis=0)
  return population


# Gennerate first population
def generate_first_generation(population_size, member_size):
  return generate_random_members(population_size, member_size)


In [ ]:
# Function to visualize poblation with utility information. Table is enumarated and ordered.
def visualize_population(members, dataframe, types, type_sizes):
  data = []

  # Create mask fot "Tipo" column
  #types = ['Recombinación', 'Mutación', 'Selección', 'Aleatorio']
  types_mask = []
  for idx, size in enumerate(type_sizes):
    for j in range(size):
      types_mask.append(types[idx])

  # Add rows with member and its information
  for idx, member in enumerate(members):
    data.append([types_mask[idx], #tipo de individuo
                 member, # array del individuo
                 fitness(member, dataframe), # distancia obtenida de la función de ajuste
                 [dataframe.iloc[member[i]][member[i+1]] for i in range(len(member)-1)], # distancias parciales obtenidas del dataframe
                 " - ".join([dataframe.index[i] for i in member])]) # nombres de municipios recorridos

  # Order rows by fitness (column 3, "Distancia(Km)")
  data = sorted(data, key= lambda member: member[2])
  indexed_data = []
  for idx, row in enumerate(data):
    indexed_row = [idx+1]
    for column in row:
      indexed_row.append(column)
    indexed_data.append(indexed_row)

  # Data is showed in columns
  print ("\n",tabulate(indexed_data, headers=["", "Tipo", "Individuo", "Distancia(Km)", "Parciales(Km)", "Ruta"]),"\n")
  return


# Function to visualize array of members with utility information.
def visualize_type(members, dataframe, order_members):
  data = []

  # Order rows by fitness is flag is True
  if order_members:
    members = np.array(sorted(members, key= lambda member: fitness(member, dataframe)))

  # Add rows with member and its information
  for idx, member in enumerate(members):
    data.append([idx+1, #index
                 member, #member
                 fitness(member, dataframe), #fitness
                 [dataframe.iloc[member[i]][member[i+1]] for i in range(len(member)-1)], #partial distances
                 " - ".join([dataframe.index[i] for i in member])]) #mapped municipalities names

  # # Data is showed in columns
  print ("\n",tabulate(data, headers=["", "Individuo", "Distancia(Km)", "Parciales(Km)", "Ruta"]),"\n")
  return


# Manage information showed for each generation
def visualize_generation(population, dataframe, generation, types, type_sizes, type_members, mask, order_members):

  print("\n*********************\n*** Generación {:2} ***\n*********************\n".format(generation))
  # Mask indicate if a partial table is showed. Type members has data for partial tables.
  if mask[0]:
    print("Mejores individuos\n******************")
    visualize_type(type_members[0], dataframe, order_members)
  if mask[1]:
    print("Recombinaciones\n**********")
    visualize_type(type_members[1], dataframe, order_members)
  if mask[2]:
    print("Mutaciones\n***************")
    visualize_type(type_members[2], dataframe, order_members)
  if mask[3]:
    print("Seleccionados\n*************")
    visualize_type(type_members[3], dataframe, order_members)
  if mask[4]:
    print("Aleatorios\n**********")
    visualize_type(type_members[4], dataframe, order_members)
  # Population is always showed
  print("Poblacion\n*********")
  visualize_population(population, dataframe, types, type_sizes)

# Hiperparámetros y cálculo de variables

In [ ]:
###############
# Hiperparams #
###############

number_of_generations = 30 # Number of populations to generate
population_size = 30 # Number of member in a poblation

prob_recombination = 0.3 # Percentage of recombinated memebers
prob_mutation = 0.4 # Percentage of mutated memebers
perc_selection_members = 0.2 # Percentage of recicled memebers

##############
# Other vars #
##############

# Mask to indicate if partial tables are showed and ordered
visualize_mask = [True, True, True, True, True] # [Selection, Recombination, Mutation, Recicled, Random]
order_masked_tables = True # Order partial tables

n_chromosomes = len(df.index) # Size of member

# Fix percentages. Use weights if 1 exceeded.
total_perc = prob_recombination + prob_mutation + perc_selection_members
if total_perc > 1:
  prob_recombination /= total_perc
  prob_mutation /= total_perc
  perc_selection_members /= total_perc

# Calculate members of each type.
size_mutation_members = round(population_size*prob_mutation)
size_recombination_members = round(population_size * prob_recombination)
size_selection_members = round(population_size * perc_selection_members)
size_random_members = population_size - size_mutation_members - size_recombination_members - size_selection_members

# Calculate neccesary selected memebers for generate new recombinated members.
size_members_to_recombine = 2
while size_recombination_members > 2*sum(range(1,size_members_to_recombine)):
  size_members_to_recombine+=1
size_members_to_recombine = max(size_members_to_recombine, size_selection_members)

# Auxiliar arrays for visualize tables. Indicates types and quantity in poblation.
sizes = [size_mutation_members, size_recombination_members, size_selection_members, size_random_members]
types = ['Mutación', 'Recombinación', 'Selección', 'Aleatorio']

# Information
print("Generaciones:", number_of_generations)
print("Población:", population_size)
print("\nMutaciones:", size_mutation_members)
print("Recombinaciones:", size_recombination_members)
print("Selección:", size_selection_members)
print("Aleatorio:", size_random_members)

Generaciones: 30
Población: 30

Mutaciones: 12
Recombinaciones: 9
Selección: 6
Aleatorio: 3


# Ejecución del algoritmo

In [ ]:
# Generate firt population
generation = 1
population = generate_first_generation(population_size, n_chromosomes)

# Visualize first poblation
print("\n*************************\n*** Población Inicial ***\n*************************\n".format(generation))
visualize_type(population, df, True)

# Generations loop
while generation < number_of_generations:

  generation += 1

  # Select best members for next generation
  selected_members = selection(population, size_members_to_recombine, df)

  # Generate recombinated members
  recombinated_members = recombinate_members(selected_members, size_recombination_members)

  # Generate mutated members
  mutated_members = mutate_generation(selected_members[0], size_mutation_members)

  # Individuos guardados o "clonados" para la próxima generación
  cloned_members = selected_members[:size_selection_members]

  # Recicled members
  random_members = generate_random_members(size_random_members, n_chromosomes)

  # Join members in a population
  population = np.concatenate((recombinated_members, mutated_members, cloned_members, random_members),axis=0)

  # Visualize of generation results
  type_members = [selected_members, recombinated_members, mutated_members, cloned_members, random_members]
  visualize_generation(population, df, generation, types, sizes, type_members, visualize_mask, order_masked_tables)

# Visualize best member found
print("\n***********************\n*** Mejor individuo ***\n***********************")
visualize_type([selected_members[0]],df, False)


*************************
*** Población Inicial ***
*************************


     Individuo                                                           Distancia(Km)  Parciales(Km)                                                                                                              Ruta
--  ----------------------------------------------------------------  ---------------  -------------------------------------------------------------------------------------------------------------------------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 1  [ 6  2  8 20 15 14  0 18  4 12 16  5  7  1  9 10 17  3 11 19 13]              661  [38.0, 25.0, 90.0, 22.0, 37.0, 11.0, 30.0, 6.0, 19.0, 19.0, 32.0, 44.0, 3.0, 57.0, 24.0, 22.0, 27.0, 40.0, 61.0, 54.0]     